In [1]:
import pandas as pd
import numpy as np
from utils.print_tree import printBTree
#from utils.rst_annotation import DiscourseUnit

import sys
sys.path.append('../')
sys.path.append('../../')
sys.path.append('../../../')

In [81]:
class DiscourseUnit:
    def __init__(self, id, left=None, right=None, text='', start=None, end=None, 
                 orig_text=None, relation=None, nuclearity=None, proba=1.):
        """
        :param int id:
        :param DiscourseUnit left:
        :param DiscourseUnit right:
        :param str text: (optional)
        :param int start: start position in original text
        :param int end: end position in original text
        :param string relation: {the relation between left and right components | 'elementary' | 'root'}
        :param string nuclearity: {'NS' | 'SN' | 'NN'}
        :param float proba: predicted probability of the relation occurrence
        """
        self.id = id
        self.left = left
        self.right = right
        self.relation = relation
        self.nuclearity = nuclearity
        self.proba = str(proba)
        self.start = start
        self.end = end

        if self.left:
            gap_counter = 0
            #while len(left.text + right.text) < len(self.text):
            #    self.text = left.text + ' ' * gap_counter + right.text
            #    gap_counter += 1
            self.start = left.start
            self.end = right.end
        
        # (1) for gold tree parsing
        """
        if orig_text:            
            self.text = orig_text[self.start:self.end].strip()
        else:
            self.text = text.strip()
        """
        # (2) ??
        
        if self.left:
            self.text = ' '.join([self.left.text, self.right.text])
        else:
            self.text = orig_text[self.start:self.end].strip()
    
    def __str__(self):
        return f"id: {self.id}\ntext: {self.text}\nrelation: {self.relation}\nleft: {self.left.text if self.left else None}\nright: {self.right.text if self.right else None}\nstart: {self.start}\nend: {self.end}"


In [82]:
def printTree(tree):
    def _(n):
        if n.relation:
            value = (n.relation, "%.2f"%(n.proba))
        else:
            value = n.text
        return str(value), n.left, n.right

    return printBTree(_) 

In [83]:
class DiscourseUnitCreator:
    def __init__(self, id):
        self.id = id
        
    def __call__(self, left_node, right_node, proba):
        self.id += 1
        return DiscourseUnit(
            id=id,
            left=left_node,
            right=right_node,
            relation=1,
            proba=proba
        )

In [84]:
#from isanlp.annotation_rst import DiscourseUnit
import pandas as pd


class RSTTreePredictor:
    def __init__(self, features_processor, relation_predictor, label_predictor):
        self.features_processor = features_processor
        self.relation_predictor = relation_predictor
        self.label_predictor = label_predictor
        if self.label_predictor:
            self.labels = self.label_predictor.classes_
        self.genre = None

    def predict_label(self, features):
        if not self.label_predictor:
            return 'relation'

        return self.label_predictor.predict(features)


class GoldTreePredictor(RSTTreePredictor):
    def __init__(self, corpus):
        RSTTreePredictor.__init__(self, None, None, None)
        self.corpus = corpus

    def extract_features(self, *args):
        return [args[0].text, args[1].text]

    def predict_pair_proba(self, features):
        def _check_snippet_pair_in_dataset(left_snippet, right_snippet):
            return ((((self.corpus.snippet_x == left_snippet) & (self.corpus.snippet_y == right_snippet)).sum(
                axis=0) != 0)
                    or ((self.corpus.snippet_y == left_snippet) & (self.corpus.snippet_x == right_snippet)).sum(
                        axis=0) != 0)

        left_snippet, right_snippet = features
        return float(_check_snippet_pair_in_dataset(left_snippet, right_snippet))

    def predict_label(self, features):
        left_snippet, right_snippet = features
        label = self.corpus[((self.corpus.snippet_x == left_snippet) & (self.corpus.snippet_y == right_snippet))].category_id.values
        if label.size == 0:
            return 'relation'
        
        return label[0]
    
    def predict_nuclearity(self, features):
        left_snippet, right_snippet = features
        nuclearity = self.corpus[((self.corpus.snippet_x == left_snippet) & (self.corpus.snippet_y == right_snippet))].order.values
        if nuclearity.size == 0:
            return '_'
        
        return nuclearity[0]


class CustomTreePredictor(RSTTreePredictor):
    def __init__(self, features_processor, relation_predictor, label_predictor=None):
        RSTTreePredictor.__init__(self, features_processor, relation_predictor, label_predictor)

    def extract_features(self, left_node: DiscourseUnit, right_node: DiscourseUnit,
                         annot_text, annot_tokens, annot_sentences, annot_postag, annot_morph, annot_lemma,
                         annot_syntax_dep_tree):
        pair = pd.DataFrame({
            'snippet_x': [left_node.text.strip()],
            'snippet_y': [right_node.text.strip()],
            #'genre': self.genre
        })

        try:
            features = self.features_processor(pair, annot_text=annot_text,
                                               annot_tokens=annot_tokens, annot_sentences=annot_sentences,
                                               annot_postag=annot_postag, annot_morph=annot_morph,
                                               annot_lemma=annot_lemma, annot_syntax_dep_tree=annot_syntax_dep_tree)
            return features
        except IndexError:
            with open('errors.log', 'w+') as f:
                f.write(str(pair.values))
                f.write(annot_text)
            return -1

    def predict_pair_proba(self, features):
        return self.relation_predictor.predict_proba(features)[0][1]

In [85]:
import numpy as np
import sys

#from isanlp.annotation_rst import DiscourseUnit


class GreedyRSTParser:
    def __init__(self, tree_predictor, forest_threshold=0.05):
        """
        :param RSTTreePredictor tree_predictor:
        :param float forest_threshold: minimum relation probability to append the pair into the tree
        """
        self.tree_predictor = tree_predictor
        self.forest_threshold = forest_threshold

    def __call__(self, edus, annot_text, annot_tokens, annot_sentences, annot_postag, annot_morph, annot_lemma,
                 annot_syntax_dep_tree, genre=None):
        """
        :param list edus: DiscourseUnit
        :param str annot_text: original text
        :param list annot_tokens: isanlp.annotation.Token
        :param list annot_sentences: isanlp.annotation.Sentence
        :param list annot_postag: lists of str for each sentence
        :param annot_lemma: lists of str for each sentence
        :param annot_syntax_dep_tree: list of isanlp.annotation.WordSynt for each sentence
        :return: list of DiscourseUnit containing each extracted tree
        """

        def to_merge(scores):
            return np.argmax(np.array(scores))

        self.tree_predictor.genre = genre

        nodes = edus
        
        for edu in nodes:
            print(edu, file=sys.stderr)
        
        max_id = edus[-1].id

        # initialize scores
        features = [
            self.tree_predictor.extract_features(nodes[i], nodes[i + 1], annot_text, annot_tokens,
                                                 annot_sentences,
                                                 annot_postag, annot_morph, annot_lemma,
                                                 annot_syntax_dep_tree)
            for i in range(len(nodes) - 1)]

        scores = [self.tree_predictor.predict_pair_proba(features[i]) for i in range(len(nodes) - 1)]
        relations = [self.tree_predictor.predict_label(features[i]) for i in range(len(nodes) - 1)]
        nuclearities = [self.tree_predictor.predict_nuclearity(features[i]) for i in range(len(nodes) - 1)]

        while len(nodes) > 2 and any([score > self.forest_threshold for score in scores]):
            # select two nodes to merge
            j = to_merge(scores)  # position of the pair in list
            
            # make the new node by merging node[j] + node[j+1]
            temp = DiscourseUnit(
                id=max_id + 1,
                left=nodes[j],
                right=nodes[j + 1],
                relation=self.tree_predictor.predict_label(features[j]),
                nuclearity=self.tree_predictor.predict_nuclearity(features[j]),
                proba=scores[j],
                text=nodes[j].text + nodes[j + 1].text  #annot_text[nodes[j].start:nodes[j+1].end]
            )
            
            print(temp, file=sys.stderr)
            
            max_id += 1

            # modify the node list
            nodes = nodes[:j] + [temp] + nodes[j + 2:]

            # modify the scores list
            if j == 0:
                features_right = self.tree_predictor.extract_features(nodes[j], nodes[j + 1],
                                                                annot_text, annot_tokens, 
                                                                annot_sentences, annot_postag,
                                                                annot_morph, annot_lemma, annot_syntax_dep_tree)
                predicted = self.tree_predictor.predict_pair_proba(features_right)

                scores = [predicted] + scores[j + 2:]
                features = [features_right] + features[j + 2:]

            elif j + 1 < len(nodes):
                features_left = self.tree_predictor.extract_features(nodes[j - 1], nodes[j], 
                                                                     annot_text, annot_tokens,
                                                                     annot_sentences, annot_postag, 
                                                                     annot_morph, annot_lemma, annot_syntax_dep_tree)
                predicted_left = self.tree_predictor.predict_pair_proba(features_left)

                features_right = self.tree_predictor.extract_features(nodes[j], nodes[j + 1], 
                                                                      annot_text, annot_tokens,
                                                                      annot_sentences, annot_postag, 
                                                                      annot_morph, annot_lemma, annot_syntax_dep_tree)
                predicted_right = self.tree_predictor.predict_pair_proba(features_right)

                scores = scores[:j - 1] + [predicted_left] + [predicted_right] + scores[j + 2:]
                features = features[:j - 1] + [features_left] + [features_right] + features[j + 2:]

            else:
                features_left = self.tree_predictor.extract_features(nodes[j - 1], nodes[j],
                                                                annot_text, annot_tokens, 
                                                                annot_sentences, annot_postag,
                                                                annot_morph, annot_lemma, annot_syntax_dep_tree)
                predicted = self.tree_predictor.predict_pair_proba(features_left)
                scores = scores[:j - 1] + [predicted]
                features = features[:j - 1] + [features_left]

        if len(scores) == 1 and scores[0] > self.forest_threshold:
            root = DiscourseUnit(
                id=max_id + 1,
                left=nodes[0],
                right=nodes[1],
                relation='root',
                proba=scores[0]
            )
            nodes = [root]

        return nodes


In [86]:
import numpy as np
import pandas as pd


def get_embeddings(embedder, X, maxlen=100):
    X_ = [text[:text.rfind('_')] for text in X.split()]
    result = np.zeros((embedder.vector_size, maxlen))

    for i in range(min(len(X_), maxlen)):
        try:
            result[i] = embedder[X_[i]]
        except KeyError:
            continue

    return result


class FeaturesExtractor:
    DROP_COLUMNS = ['snippet_x', 'snippet_y', 'snippet_x_tmp', 'snippet_y_tmp', 'postags_x', 'postags_y']

    def __init__(self, processor, scaler=None, categorical_cols=None, one_hot_encoder=None, label_encoder=None):
        self.processor = processor
        self.scaler = scaler
        self._categorical_cols = categorical_cols
        self.one_hot_encoder = one_hot_encoder
        self.label_encoder = label_encoder

    def __call__(self, df, annot_text, annot_tokens, annot_sentences, annot_postag, annot_morph, annot_lemma, annot_syntax_dep_tree):
        X = self.processor(df, annot_text, annot_tokens, annot_sentences, annot_postag, annot_morph, annot_lemma, annot_syntax_dep_tree)
        X = X.drop(columns=self.DROP_COLUMNS)

        if self._categorical_cols:
            if self.label_encoder:
                X[self._categorical_cols] = X[self._categorical_cols].apply(lambda col: self.label_encoder.fit_transform(col))

            if self.one_hot_encoder:
                X_ohe = self.one_hot_encoder.transform(X[self._categorical_cols].values)
                X_ohe = pd.DataFrame(X_ohe, X.index, columns=self.one_hot_encoder.get_feature_names(self._categorical_cols))

                X = X.join(
                    pd.DataFrame(X_ohe, X.index).add_prefix('cat_'), how='right'
                ).drop(columns=self._categorical_cols).drop(columns=self.DROP_COLUMNS)

        if self.scaler:
            return pd.DataFrame(self.scaler.transform(X.values), index=X.index, columns=X.columns)

        return X


## Gold tree parsing example 

In [87]:
def extr_pairs(tree):
    pp = []
    if tree.left:
        pp.append([tree.left.text, tree.right.text, tree.relation])
        pp += extr_pairs(tree.left)
        pp += extr_pairs(tree.right)
    return pp

def extr_pairs_forest(forest):
    pp = []
    for tree in forest:
        pp += extr_pairs(tree)
    return pp

In [88]:
from utils.train_test_split import split_data

train, test = split_data('data/', 0.2)

news in train: 0.38114754098360654,	in test: 0.3114754098360656
ling in train: 0.1680327868852459,	in test: 0.14754098360655737
comp in train: 0.13114754098360656,	in test: 0.22950819672131148


In [89]:
test[2]

'data/blogs_60.edus'

In [107]:
%%time

from tqdm import tqdm_notebook as tqdm
from utils.file_reading import read_edus, read_gold, read_annotation

cache = {}
for file in tqdm(test[:3]):
    filename = '.'.join(file.split('.')[:-1])
    edus = read_edus(filename)
    gold = read_gold(filename)
    annot = read_annotation(filename)
    
    _edus = []
    last_end = 0
    for max_id in range(len(edus)):
        start = len(annot['text'][:last_end]) + annot['text'][last_end:].find(edus[max_id])
        end = start + len(edus[max_id])
        temp = DiscourseUnit(
                id=max_id,
                left=None,
                right=None,
                relation='edu',
                start=start,
                end=end,
                orig_text=annot['text'],
                proba=1.,
                #text=edus[max_id]  #annot_text[nodes[j].start:nodes[j+1].end]
            )
        _edus.append(temp)
        last_end = end

    parser = GreedyRSTParser(GoldTreePredictor(gold), forest_threshold=0.)
    parsed = parser(_edus, annot['text'], annot['tokens'], annot['sentences'],
                    annot['postag'], annot['morph'], annot['lemma'], annot['syntax_dep_tree'])
    
    parsed_pairs = pd.DataFrame(extr_pairs_forest(parsed), columns=['snippet_x', 'snippet_y', 'category_id'])
    cache[filename] = (parsed_pairs, gold)

/opt/.pyenv/versions/3.7.4/lib/python3.7/site-packages/ipykernel_launcher.py:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """


id: 0
text: Омер Бичер
relation: edu
left: None
right: None
start: 0
end: 10
id: 1
text: ПОСЛОВИЦЫ И ПОГОВОРКИ С АНИМАЛИСТИЧЕСКИМИ ОБРАЗАМИ ВО ФРАЗЕОЛОГИЧЕСКИХ СЛОВАРЯХ
relation: edu
left: None
right: None
start: 11
end: 90
id: 2
text: Адрес статьи: www.gramota.netmaterials27201511 -2735.html
relation: edu
left: None
right: None
start: 91
end: 148
id: 3
text: Источник Филологические науки. Вопросы теории и практики
relation: edu
left: None
right: None
start: 149
end: 205
id: 4
text: Тамбов: Грамота, 2015. № 11(53): в 3-х ч. Ч. II. C. 136-138. ISSN 1997-2911.
relation: edu
left: None
right: None
start: 206
end: 282
id: 5
text: Адрес журнала: www.gramota.neteditions2.html
relation: edu
left: None
right: None
start: 283
end: 327
id: 6
text: Содержание данного номера журнала: www .gramota.netmate rials2201511-2
relation: edu
left: None
right: None
start: 328
end: 398
id: 7
text: © Издательство "Грамота"
relation: edu
left: None
right: None
start: 399
end: 423
id: 8
text: Информация о возмо

id: 142
text: на синхронном уровне решить проблему тождества (различение инвариантных и вариантных отношений), а на диахроническом уровне установить зависимость между элементами системы и их потенциями исторического развития» [18, с. 161].
relation: joint_m
left: на синхронном уровне решить проблему тождества (различение инвариантных и вариантных отношений),
right: а на диахроническом уровне установить зависимость между элементами системы и их потенциями исторического развития» [18, с. 161].
start: 1978
end: 2203
id: 143
text: Системность и операционный характер модели мира дают возможность на синхронном уровне решить проблему тождества (различение инвариантных и вариантных отношений), а на диахроническом уровне установить зависимость между элементами системы и их потенциями исторического развития» [18, с. 161].
relation: solutionhood_r
left: Системность и операционный характер модели мира дают возможность
right: на синхронном уровне решить проблему тождества (различение инвариантных и

end: 8972
id: 190
text: Сборник А. А. Епифанова «Русские пословицы и поговорки и их турецкие аналоги (Turk atasozleri ve rus karsiliklari)» [8] содержит 900 турецких пословиц и их русские эквиваленты или аналоги, расположенные в алфавитном порядке (близкие по смыслу русские пословицы, в отдельных случаях - поговорки и крылатые выражения). В словарной статье также приводятся варианты компонентов турецких пословиц или их факультативные компоненты. Курсивом даются пояснения автора.
relation: elaboration_r
left: Сборник А. А. Епифанова «Русские пословицы и поговорки и их турецкие аналоги (Turk atasozleri ve rus karsiliklari)» [8] содержит 900 турецких пословиц и их русские эквиваленты или аналоги, расположенные в алфавитном порядке (близкие по смыслу русские пословицы, в отдельных случаях - поговорки и крылатые выражения).
right: В словарной статье также приводятся варианты компонентов турецких пословиц или их факультативные компоненты. Курсивом даются пояснения автора.
start: 8513
end: 89

id: 189
text: ФБ гудит. Подопечных фонда "Волонтеры в помощь детям-сиротам" лишили родительских прав на четверых детей IMG
relation: cause_r
left: ФБ гудит.
right: Подопечных фонда "Волонтеры в помощь детям-сиротам" лишили родительских прав на четверых детей IMG
start: 92
end: 200
id: 190
text: Фонд "Волонтеры в помощь детям-сиротам" помогает семье с 2012 года и даже сделал в 2013 ремонт в квартире.
relation: sequence_m
left: Фонд "Волонтеры в помощь детям-сиротам" помогает семье с 2012 года
right: и даже сделал в 2013 ремонт в квартире.
start: 201
end: 307
id: 191
text: ФБ гудит. Подопечных фонда "Волонтеры в помощь детям-сиротам" лишили родительских прав на четверых детей IMG Фонд "Волонтеры в помощь детям-сиротам" помогает семье с 2012 года и даже сделал в 2013 ремонт в квартире.
relation: preparation_r
left: ФБ гудит. Подопечных фонда "Волонтеры в помощь детям-сиротам" лишили родительских прав на четверых детей IMG
right: Фонд "Волонтеры в помощь детям-сиротам" помогает семье с 201

end: 2436
id: 236
text: Автор этого комментария дал мне его на условиях анонимности, хотя живет за несколько сотен километров от Москвы.
relation: concession_r
left: Автор этого комментария дал мне его на условиях анонимности,
right: хотя живет за несколько сотен километров от Москвы.
start: 2376
end: 2488
id: 237
text: Автор этого комментария дал мне его на условиях анонимности, хотя живет за несколько сотен километров от Москвы. Но он все равно опасается как за свою безопасность, так и за жизнь своих животных.
relation: contrast_m
left: Автор этого комментария дал мне его на условиях анонимности, хотя живет за несколько сотен километров от Москвы.
right: Но он все равно опасается как за свою безопасность, так и за жизнь своих животных.
start: 2376
end: 2571
id: 238
text: Что же, буду первой, кто заговорит. Автор этого комментария дал мне его на условиях анонимности, хотя живет за несколько сотен километров от Москвы. Но он все равно опасается как за свою безопасность, так и за жизнь 

id: 283
text: - Мое мнение - данная передержка была очень дешевой, фактически "за корм", но деньги на эту передержку кураторы животных собирали по полной программе. Волонтеры от зоозащиты (далее я буду называть их только зоошиза - потому что только шиза могла отправлять животных на данную передержку) пользовались психическими особенностями женщины. Эти психические особенности имеют название - хордерство. Собирательство животных. Кстати данный "таракан" в голове у хозяйки "передержки" подтверждают и волонтеры фонда "Отказники", которые курировали семью. Правда в своих текстах в защиту семьи они представляют Надежду как сердобольную спасительницу. Но нет. Это не спасение и не помощь животным. Это издевательство над ними, над их природой. Это психическое заболевание - брать в дом больше животных, чем ты можешь обеспечить не только едой, но и пространством, и вниманием. В США и Западной Европе собак бы просто изъяли у человека с полицией. И после следили бы , чтобы данная семья никогда не 

id: 333
text: За правами детей хоть и со скрипом, но следят врачи и органы опеки. Так что если только собирать деньги, но не лечить ребенка и не кормить его, рано или поздно на пороге нарисуется чиновник и начнет задавать неприятные вопросы.
relation: cause_r
left: За правами детей хоть и со скрипом, но следят врачи и органы опеки.
right: Так что если только собирать деньги, но не лечить ребенка и не кормить его, рано или поздно на пороге нарисуется чиновник и начнет задавать неприятные вопросы.
start: 7932
end: 8159
id: 334
text: Собаки и кошки никому не интересны, пока их дерьмо не начнет протекать к соседям.
relation: condition_r
left: Собаки и кошки никому не интересны,
right: пока их дерьмо не начнет протекать к соседям.
start: 8160
end: 8241
id: 335
text: И, в отличие от "выгодной деточки" выгодных собачек можно насобирать по обочинам в любой момент, если старые сдохнут.
relation: condition_r
left: И, в отличие от "выгодной деточки" выгодных собачек можно насобирать по обочинам в


CPU times: user 2.67 s, sys: 83.5 ms, total: 2.75 s
Wall time: 2.64 s


id: 58
text: Желающие могут или выкупить их по себестоимости, или проработать на предприятии 20 лет,
relation: joint_m
left: Желающие могут или выкупить их по себестоимости,
right: или проработать на предприятии 20 лет,
start: 248
end: 335
id: 59
text: чтобы получить квартиры в собственность без какой-либо оплаты:
relation: condition_r
left: чтобы получить квартиры в собственность
right: без какой-либо оплаты:
start: 336
end: 398
id: 60
text: Желающие могут или выкупить их по себестоимости, или проработать на предприятии 20 лет, чтобы получить квартиры в собственность без какой-либо оплаты:
relation: purpose_r
left: Желающие могут или выкупить их по себестоимости, или проработать на предприятии 20 лет,
right: чтобы получить квартиры в собственность без какой-либо оплаты:
start: 248
end: 398
id: 61
text: Желающие могут или выкупить их по себестоимости, или проработать на предприятии 20 лет, чтобы получить квартиры в собственность без какой-либо оплаты: https:news.ngs.ruarticles66093778


In [95]:
import json

#filename = 'rst_pairs/news_55'
#filename = 'rst_pairs/news_13'
filename = 'data/sci.comp_26'
edus = read_edus(filename)
gold = read_gold(filename)
annot = read_annotation(filename)

_edus = []
last_end = 0
for max_id in range(len(edus)):
    start = len(annot['text'][:last_end]) + annot['text'][last_end:].find(edus[max_id])
    end = start + len(edus[max_id])
    temp = DiscourseUnit(
            id=max_id,
            left=None,
            right=None,
            relation='edu',
            start=start,
            end=end,
            orig_text=annot['text'],
            proba=1.,
            #text=edus[max_id]  #annot_text[nodes[j].start:nodes[j+1].end]
        )
    _edus.append(temp)
    last_end = end

In [98]:
parser = GreedyRSTParser(GoldTreePredictor(gold), forest_threshold=0.)
parsed = parser(_edus, annot['text'], annot['tokens'], annot['sentences'],
                annot['postag'], annot['morph'], annot['lemma'], annot['syntax_dep_tree'])

id: 0
text: ﻿ПРИБОРЫ И СИСТЕМЫ АВТОМАТИЧЕСКОГО УПРАВЛЕНИЯ
relation: edu
left: None
right: None
start: 0
end: 45
id: 1
text: УДК 62-50
relation: edu
left: None
right: None
start: 46
end: 55
id: 2
text: Д. С. Кабанов, Б. А. Крашенинников
relation: edu
left: None
right: None
start: 56
end: 90
id: 3
text: УПРАВЛЕНИЕ ТРАЕКТОРИЕЙ ДВИЖЕНИЯ АВТОМОБИЛЯ С ИСПОЛЬЗОВАНИЕМ АЛГОРИТМА ПОСЛЕДОВАТЕЛЬНОЙ ОПТИМИЗАЦИИ
relation: edu
left: None
right: None
start: 91
end: 189
id: 4
text: Рассмотрена задача оптимального управления траекторией движения автомобиля в горизонтальной плоскости.
relation: edu
left: None
right: None
start: 190
end: 292
id: 5
text: Решение сформировано с использованием алгоритма последовательной оптимизации по иерархии целевых функционалов.
relation: edu
left: None
right: None
start: 293
end: 403
id: 6
text: Представлены результаты численного моделирования.
relation: edu
left: None
right: None
start: 404
end: 453
id: 7
text: Ключевые слова: автомобиль, оптимизация управления, динамик

id: 106
text: Использование критерия А. А. Красовского приводит к компромиссному результату по терминальным показателям точности [1].
relation: cause_r
left: Использование критерия А. А. Красовского приводит
right: к компромиссному результату по терминальным показателям точности [1].
start: 776
end: 895
id: 107
text: целесообразно использовать алгоритм последовательной оптимизации по иерархии критериев качества [2-4], который можно рассматривать как управление по критерию А. А. Красовского с заданным ограничением терминального типа.
relation: elaboration_r
left: целесообразно использовать алгоритм последовательной оптимизации по иерархии критериев качества [2-4],
right: который можно рассматривать как управление по критерию А. А. Красовского с заданным ограничением терминального типа.
start: 988
end: 1207
id: 108
text: При необходимости повышения требований к углу поворота траектории в конечный момент времени целесообразно использовать алгоритм последовательной оптимизации по иерархии 

## Evaluation 

In [118]:
from utils.evaluation import metric_parseval, extr_pairs, extr_pairs_forest, _check_snippet_pair_in_dataset, _not_parsed_as_in_gold

In [119]:
parsed_pairs = pd.DataFrame(extr_pairs_forest(parsed), columns=['snippet_x', 'snippet_y', 'category_id'])

In [120]:
parsed_pairs.shape, gold.shape

((48, 3), (49, 7))

In [121]:
errors = _not_parsed_as_in_gold(parsed_pairs, gold)

In [122]:
def find_edu_number(edus, error):
    for i, edu in enumerate(edus):
        if error[2].find(edu) > -1:
            yield i

In [123]:
true_pos = []
all_parsed = []
all_gold = []
fnames = []

c_true_pos, c_all_parsed, c_all_gold = metric_parseval(parsed_pairs, gold)
true_pos.append(c_true_pos)
all_parsed.append(c_all_parsed)
all_gold.append(c_all_gold)

recall = sum(true_pos) / sum(all_gold)
print('Recall: ', recall)

precision = sum(true_pos) / sum(all_parsed)
print('Precision:', precision)

f1 = 2 * precision * recall / (precision + recall)
print('F1:', f1)
    
aa = pd.DataFrame({'true_pos': true_pos, 'all_parsed': all_parsed, 'all_gold': all_gold})
aa['recall'] = aa.true_pos / aa.all_gold
aa['precision'] = aa.true_pos / aa.all_parsed
aa['f1'] = aa.recall * aa.precision * 2 / (aa.precision + aa.recall)

aa.sort_values('f1')

Recall:  1.0
Precision: 1.0
F1: 1.0


,true_pos,all_parsed,all_gold,recall,precision,f1
0,48,48,48,1.0,1.0,1.0


# Evaluation (Parser)

In [ ]:
from utils.features_processor_default import FeaturesProcessor

binary_classifier_model_path = 'structure_predictor/'

In [ ]:
%%time

features_processor = FeaturesProcessor(model_dir_path='models', verbose=True)

In [ ]:
import pickle

scaler = pickle.load(open(binary_classifier_model_path + 'scaler.pkl', 'rb'))
#categorical_cols = pickle.load(open(binary_classifier_model_path + 'categorical_cols.pkl', 'rb'))
#ohe = pickle.load(open(binary_classifier_model_path + 'one_hot_encoder.pkl', 'rb'))
#le = pickle.load(open(binary_classifier_model_path + 'label_encoder.pkl', 'rb'))
binary_classifier = pickle.load(open(binary_classifier_model_path + 'model.pkl', 'rb'))

features_extractor = FeaturesExtractor(features_processor, scaler)

In [ ]:
predictor = CustomTreePredictor(features_extractor, binary_classifier, label_predictor=None)

In [ ]:
parser = GreedyRSTParser(predictor, forest_threshold=0.1)

In [ ]:
%%time

from tqdm import tqdm_notebook as tqdm

cache = {}
for file in tqdm(test[:1]):
    filename = ''.join(file.split('.')[:-1])
    edus = read_edus(filename)
    gold = read_gold(filename)
    annot = read_annotation(filename)
    
    _edus = []
    last_end = 0
    for max_id in range(len(edus) - 1):
        start = len(annot['text'][:last_end]) + annot['text'][last_end:].find(edus[max_id])
        end = start + len(edus[max_id])
        temp = DiscourseUnit(
                id=max_id,
                left=None,
                right=None,
                relation='edu',
                start=start,
                end=end,
                orig_text=annot['text'],
                proba=1.,
                #text=edus[max_id]  #annot_text[nodes[j].start:nodes[j+1].end]
            )
        _edus.append(temp)
        last_end = end

    parsed = parser(_edus, 
                    annot['text'], 
                    annot['tokens'], 
                    annot['sentences'], 
                    annot['postag'], 
                    annot['morph'], 
                    annot['lemma'], 
                    annot['syntax_dep_tree'], 
                    genre=filename.split('_')[0])
    
    parsed_pairs = pd.DataFrame(extr_pairs_forest(parsed), columns=['snippet_x', 'snippet_y', 'category_id'])
    cache[filename] = (parsed_pairs, gold)

In [ ]:
len(edus)

# Evaluation (Gold)

In [ ]:
import glob
import os
from tqdm import tqdm_notebook as tqdm

cache = {}
for file in tqdm(glob.glob('rst_pairs/*.edus')):
    filename = file.replace('.edus', '')
    edus = read_edus(filename)
    gold = read_gold(filename)
    gold = gold.sort_values('snippet_y').drop_duplicates(subset=['snippet_y'])
    annot = read_annotation(filename)
    _edus = []
    last_end = 0
    for max_id in range(len(edus)):
        start = len(annot['text'][:last_end]) + annot['text'][last_end:].find(edus[max_id])
        end = start + len(edus[max_id])
        temp = DiscourseUnit(
                id=max_id,
                left=None,
                right=None,
                relation='edu',
                start=start,
                end=end,
                orig_text=annot['text'],
                proba=1.,
                #text=edus[max_id]  #annot_text[nodes[j].start:nodes[j+1].end]
            )
        _edus.append(temp)
        last_end = end
    
    parser = GreedyRSTParser(GoldTreePredictor(gold), forest_threshold=0.)
    parsed = parser(_edus)
    parsed_pairs = pd.DataFrame(extr_pairs_forest(parsed), columns=['snippet_x', 'snippet_y', 'category_id'])
    cache[filename] = (parsed_pairs, gold)

In [ ]:
filenames = []
true_pos = []
all_parsed = []
all_gold = []

for key, value in cache.items():
    c_true_pos, c_all_parsed, c_all_gold = metric_parseval(value[0], value[1])
    filenames.append(key)
    true_pos.append(c_true_pos)
    all_parsed.append(c_all_parsed)
    all_gold.append(c_all_gold)
    
results = pd.DataFrame({'filename': filenames, 
                    'true_pos': true_pos,
                    'all_parsed': all_parsed,
                    'all_gold': all_gold})

In [ ]:
results['recall'] = results['true_pos'] / results['all_gold']
results['precision'] = results['true_pos'] / results['all_parsed']
results['F1'] = 2 * results['precision'] * results['recall'] / (results['precision'] + results['recall'])

In [ ]:
results[results['filename'].str.contains('comp')].F1.mean()

In [ ]:
results[results['filename'].str.contains('ling')].F1.mean()

In [ ]:
results[results['filename'].str.contains('news')].F1.mean()